In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import ast

In [3]:
df_path = "data/newsela/articles.csv"

# Import Data

In [13]:
base_url = 'https://newsela.com'
url = base_url + '/articles/#/rule/latest'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)


In [21]:
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome()

browser.get("https://newsela.com/articles/#/rule/latest")
time.sleep(1)

elem = browser.find_element_by_tag_name("body")

no_of_pagedowns = 20

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)
    no_of_pagedowns-=1

post_elems = browser.find_elements_by_class_name("header-image-container")

for post in post_elems:
    print(post.text)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [19]:
filters

[]

In [134]:
def import_britishcouncil_articles():
    base_url = 'https://newsela.com'
    url = base_url + '/articles/#/rule/latest'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    
    soup = BeautifulSoup(page.content, 'html.parser')
    filters = soup.select("div.views-row a")
    
    articles = []
    art_set = set()
    print("Select Articles...")
    for article in filters:
        article_href = article.get_attribute_list("href")[0]
        article_name = article_href.split("/")[-1][:-2]
        article_level = article_href[-1]
        if "1" == article_level and article_name not in art_set: 
            articles += [(article_href[:-2],article_name)]
            art_set.update([article_name])
            
    texts = []
    print("Get Texts...")
    for article in tqdm(articles):
        text_levels = []
        for level in ["-1","-2","-3"]:
            article_page = requests.get(base_url + article[0] + level, headers=headers)
            article_soup = BeautifulSoup(article_page.content, 'html.parser')
            text_parts = [text_part.get_text() for text_part in article_soup.select("div.field-name-body p")]
            text_levels += [text_parts]
        texts += [(article[1],*text_levels)]
        
    return pd.DataFrame(data=texts,columns=["article_name","raw1","raw2","raw3"])

In [165]:
df = import_britishcouncil_articles()

  0%|          | 0/22 [00:00<?, ?it/s]

Select Articles...
Get Texts...


100%|██████████| 22/22 [00:10<00:00,  2.15it/s]


In [166]:
df.head()

,article_name,raw1,raw2,raw3
0,walk-forest-level,[Going through the forest is my favourite part...,[Going through the forest is my favourite part...,[Going through the forest is my favourite part...
1,amazing-adventurers-level,[Do you ever dream about climbing Mount Everes...,[Have you ever dreamt of climbing Mount Everes...,[Have you ever dreamt of climbing Mount Everes...
2,animals-city-level,"[One night in December 2011, a bear came into ...","[Recently, there have been many reports in new...","[Small animals like birds, squirrels, mice and..."
3,bully-level,[Kay got another message as she was leaving fo...,[Kay got another message as she was leaving fo...,[Kay got another message as she was leaving fo...
4,cheat-level,[Mo was waiting outside her class. She was fee...,[Mo was waiting in the corridor outside her cl...,[Mo was waiting in the corridor with her class...


In [183]:
def save_df(path,df):
    df.to_csv(path,sep="|",index=False)

In [167]:
save_df(df_path,df)

# Prepare Data

In [140]:
def load_df(path,list_cols):
    df = pd.read_csv(path,sep="|")
    df[list_cols] = df[list_cols].applymap(
    lambda x: ast.literal_eval(x))
    return df

In [175]:
df = load_df(df_path,["raw1","raw2","raw3"])
df.head()

,article_name,raw1,raw2,raw3
0,walk-forest-level,[Going through the forest is my favourite part...,[Going through the forest is my favourite part...,[Going through the forest is my favourite part...
1,amazing-adventurers-level,[Do you ever dream about climbing Mount Everes...,[Have you ever dreamt of climbing Mount Everes...,[Have you ever dreamt of climbing Mount Everes...
2,animals-city-level,"[One night in December 2011, a bear came into ...","[Recently, there have been many reports in new...","[Small animals like birds, squirrels, mice and..."
3,bully-level,[Kay got another message as she was leaving fo...,[Kay got another message as she was leaving fo...,[Kay got another message as she was leaving fo...
4,cheat-level,[Mo was waiting outside her class. She was fee...,[Mo was waiting in the corridor outside her cl...,[Mo was waiting in the corridor with her class...


In [179]:
replace_strings = ["\xa0\n\t","\n\t","\xa0"]

In [180]:
def clean_data(replace_strings,raw_text):
    for string in replace_strings:
        raw_text = [part.replace(string," ") for part in raw_text]
    return raw_text

In [182]:
df[["cleaned1","cleaned2","cleaned3"]] = df[["raw1","raw2","raw3"]].applymap(lambda x: clean_data(replace_strings,x))

In [185]:
df_cleaned_path = "data/british_council/cleaned_articles.csv"
save_df(df_cleaned_path,df[["article_name","cleaned1","cleaned2","cleaned3"]])

# Selenium

In [1]:
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [45]:
browser = webdriver.Chrome()

browser.get("https://newsela.com/articles/#/rule/latest?languages=en&required_tag=cp:tribune-content-agency")

time.sleep(600)

post_elems = browser.find_elements_by_class_name("header-image-container")

for post in post_elems:
    print(post.get_property('href'))

https://newsela.com/read/elem-mountain-lion-cubs/id/44504
https://newsela.com/read/elem-manta-ray-nursery-discovery/id/44447
https://newsela.com/read/Disney-princesses-get-successful-jobs/id/44519
https://newsela.com/read/letters-of-sorrow-joy-win-award/id/44118
https://newsela.com/read/parkland-students-tony-awards/id/44221
https://newsela.com/read/less-frequent-storms-cause-more-damage/id/44147
https://newsela.com/read/elem-chess-lady-visits-students/id/44317
https://newsela.com/read/students-united-nations/id/43980
https://newsela.com/read/elem-lettuce-club/id/44163
https://newsela.com/read/pets-hiking-cat/id/41278
https://newsela.com/read/USC-youngest-student-overcomes-autism/id/43444
https://newsela.com/read/boys-girls-club-first-gen-college-students/id/43520
https://newsela.com/read/pro-con-college-athletes-paid/id/43396
https://newsela.com/read/elem-falcon-chicks-webcam/id/43662
https://newsela.com/read/teens-sports-drinks/id/43441
https://newsela.com/read/students-respond-to-ha

https://newsela.com/read/antarctica-iceberg/id/32829
https://newsela.com/read/parks-prescription/id/32676
https://newsela.com/read/hijab-college-golfer/id/32644
https://newsela.com/read/elem-100-year-old-painter/id/32126
https://newsela.com/read/elem-teen-animal-youtube-channel/id/32157
https://newsela.com/read/elem-new-jersey-state-germ/id/32028
https://newsela.com/read/grizzly-bears-endangered-list/id/32272
https://newsela.com/read/handwriting-champion/id/31945
https://newsela.com/read/study-teenagers-emotional-health-social-media/id/31749
https://newsela.com/read/elem-student-boat-racing/id/31756
https://newsela.com/read/financial-literacy-procon/id/31585
https://newsela.com/read/facebook-constituent/id/31663
https://newsela.com/read/walk-run-club/id/31488
https://newsela.com/read/refugee-scouts/id/31329
https://newsela.com/read/esports-high-school/id/31213
https://newsela.com/read/marley-dias-1000blackgirlbooks/id/31206
https://newsela.com/read/japan-disaster-theme-park/id/31098
ht

https://newsela.com/read/hiroshima-okonomiyaki/id/21921
https://newsela.com/read/charlotte-protests/id/21990
https://newsela.com/read/mammoth-skull-discovery/id/21877
https://newsela.com/read/teen-runner-michelle-cross/id/21878
https://newsela.com/read/cup-noodles-healthy/id/21711
https://newsela.com/read/us-mexico-schools/id/21858
https://newsela.com/read/nasa-twins-study/id/21573
https://newsela.com/read/nativeamerican-students/id/21565
https://newsela.com/read/procon-self-driving-cars-future/id/21480
https://newsela.com/read/ancient-cone-fossils/id/21430
https://newsela.com/read/grizzly-bear-ca-reintroduction/id/21446
https://newsela.com/read/female-pilot-arlington/id/21474
https://newsela.com/read/jupiter-new-images/id/21375
https://newsela.com/read/north-dakota-pipeline/id/21233
https://newsela.com/read/lucky-fins/id/21212
https://newsela.com/read/lucy-fossil-mystery/id/21188
https://newsela.com/read/family-baking-business/id/21146
https://newsela.com/read/trump-deportations-check

https://newsela.com/read/fukushima-robots/id/15712
https://newsela.com/read/hoverboards-safetytesting/id/15638
https://newsela.com/read/processed-foods/id/15656
https://newsela.com/read/ocean-noise/id/15512
https://newsela.com/read/owl-war/id/15503
https://newsela.com/read/americangirl-melody/id/15461
https://newsela.com/read/jellyfish-app/id/15347
https://newsela.com/read/texmex-colonialpageant/id/15256
https://newsela.com/read/selfdrivingcars-procon/id/15469
https://newsela.com/read/stlouis-lead/id/15431
https://newsela.com/read/homeless-tinyhomes/id/15352
https://newsela.com/read/univision-trump/id/15321
https://newsela.com/read/mindcontrolled-prosthetic/id/15230
https://newsela.com/read/superheroes-girls/id/15278
https://newsela.com/read/nancyreagan-obituary/id/15545
https://newsela.com/read/defundsanctuaries-procon/id/15282
https://newsela.com/read/paintball-game/id/15262
https://newsela.com/read/pets-stemcelltherapy/id/15235
https://newsela.com/read/target-genderneutral/id/15172


https://newsela.com/read/teen-republican/id/12612
https://newsela.com/read/clinton-youngimmigrant/id/12582
https://newsela.com/read/mcdonalds-eggs/id/12323
https://newsela.com/read/iran-nuclear/id/12542
https://newsela.com/read/exoticpets-laws/id/12294
https://newsela.com/read/drones-agriculture/id/12183
https://newsela.com/read/lego-castle/id/12248
https://newsela.com/read/telemedicine-growth/id/12158
https://newsela.com/read/rodeo-horse/id/12222
https://newsela.com/read/vw-dealers/id/12303
https://newsela.com/read/cdc-fastfood/id/12034
https://newsela.com/read/ahmed-clockteacher/id/12238
https://newsela.com/read/microsoft-africa/id/12027
https://newsela.com/read/standing-desks/id/12110
https://newsela.com/read/elnino-snakes/id/12008
https://newsela.com/read/uscuba-farmers/id/12065
https://newsela.com/read/exercise-brainbenefits/id/9627
https://newsela.com/read/africa-outlook/id/12163
https://newsela.com/read/tibet-princess/id/12071
https://newsela.com/read/syria-chicago/id/12004
http

...

https://newsela.com/read/longterm-englishlearners/id/6733
https://newsela.com/read/diverse-congress/id/6823
https://newsela.com/read/utah-speedlimits/id/6496
https://newsela.com/read/beaver-drought/id/6752
https://newsela.com/read/surfer-shark/id/6394
https://newsela.com/read/navajo-coal/id/6489
https://newsela.com/read/soccer-club/id/5718
https://newsela.com/read/afghan-hotdog/id/6484
https://newsela.com/read/airasia-crash/id/6798
https://newsela.com/read/hungarian-shopping/id/6493
https://newsela.com/read/doctor-dangerousneighborhood/id/6360
https://newsela.com/read/starving-kids/id/6467
https://newsela.com/read/teen-jobs/id/6434
https://newsela.com/read/sony-nkorea/id/6730
https://newsela.com/read/dairy-campaign/id/6450
https://newsela.com/read/india-coffee/id/6412
https://newsela.com/read/cuba-us/id/6505
https://newsela.com/read/solarenergy-kids/id/6348
https://newsela.com/read/distracted-walkers/id/6440
https://newsela.com/read/toy-injuries/id/6421
https://newsela.com/read/agtech-

https://newsela.com/read/germany-forger/id/5162
https://newsela.com/read/china-textiles/id/5129
https://newsela.com/read/gargantuan-dino/id/5121
https://newsela.com/read/sleepy-students/id/5124
https://newsela.com/read/ebola-response/id/5157
https://newsela.com/read/sapphire-apple/id/5135
https://newsela.com/read/mummies-age/id/4907
https://newsela.com/read/physics-monet/id/5106
https://newsela.com/read/wolves-yawning/id/5060
https://newsela.com/read/flights-diverted/id/5090
https://newsela.com/read/prison-design/id/5006
https://newsela.com/read/police-cameras/id/5078
https://newsela.com/read/deathvalley-rocks/id/5027
https://newsela.com/read/mustangs-northcarolina/id/5013
https://newsela.com/read/conservation-careers/id/5030
https://newsela.com/read/israelischool-children/id/5069
https://newsela.com/read/sleepy-schooltimes/id/4980
https://newsela.com/read/uzi-kids/id/5061
https://newsela.com/read/headdress-twitter/id/5016
https://newsela.com/read/bacon-priceincrease/id/4986
https://ne

https://newsela.com/read/tiananmen-square/id/3640
https://newsela.com/read/horseshoe-crabs/id/3670
https://newsela.com/read/afghan-childlaborers/id/3585
https://newsela.com/read/cat-apathy/id/3610
https://newsela.com/read/last-whalers/id/3553
https://newsela.com/read/india-cheating/id/3488
https://newsela.com/read/wheelchair-rugby/id/3541
https://newsela.com/read/teen-stowaway/id/3597
https://newsela.com/read/earthlike-planet/id/3530
https://newsela.com/read/sabbath-debate/id/3467
https://newsela.com/read/garimara-obituary/id/3567
https://newsela.com/read/corporal-punishment/id/3501
https://newsela.com/read/digital-resistance/id/3509
https://newsela.com/read/school-nogrades/id/3438
https://newsela.com/read/plane-submarine/id/3463
https://newsela.com/read/monarch-crisis/id/3427
https://newsela.com/read/ukrainian-eggs/id/3464
https://newsela.com/read/mentalhealth-schoolviolence/id/3426
https://newsela.com/read/syria-aleppo/id/3422
https://newsela.com/read/blackreporter-whitehouse/id/3450

https://newsela.com/read/library-technology/id/2393
https://newsela.com/read/doctor-shortage/id/2347
https://newsela.com/read/plastic-ocean/id/2308
https://newsela.com/read/smoking-increase/id/2375
https://newsela.com/read/china-space/id/2364
https://newsela.com/read/usa-population/id/2298
https://newsela.com/read/prison-newspaper/id/2351
https://newsela.com/read/electriccars-procon/id/1897
https://newsela.com/read/la-ipads/id/2333
https://newsela.com/read/nasa-whitehouse/id/2359
https://newsela.com/read/neanderthal-organization/id/2302
https://newsela.com/read/college-recruiters/id/2313
https://newsela.com/read/virus-bacteria/id/2332
https://newsela.com/read/headlice-hysteria/id/2259
https://newsela.com/read/invasive-fish/id/2287
https://newsela.com/read/arizona-immigration/id/2227
https://newsela.com/read/grades-homework/id/2289
https://newsela.com/read/business-veterans/id/2199
https://newsela.com/read/tribe-marijuana/id/2283
https://newsela.com/read/lizard-breathing/id/2254
https:/

https://newsela.com/read/colorado-guns/id/1093
https://newsela.com/read/chinese-teachers/id/1051
https://newsela.com/read/football-helmets/id/1033
https://newsela.com/read/voting-ID/id/1056
https://newsela.com/read/yupik-artifacts/id/953
https://newsela.com/read/energydrinks-congress/id/1019
https://newsela.com/read/deported-migrants/id/1013
https://newsela.com/read/alaska-mine/id/996
https://newsela.com/read/school-fourday/id/1001
https://newsela.com/read/syria-obama/id/1025
https://newsela.com/read/mexico-racism/id/934
https://newsela.com/read/teen-employment/id/1015
https://newsela.com/read/mural-ban/id/980
https://newsela.com/read/math-violence/id/936
https://newsela.com/read/syria-refugees/id/1007
https://newsela.com/read/parks-lines/id/840
https://newsela.com/read/native-seeds/id/921
https://newsela.com/read/restaurant-strike/id/982
https://newsela.com/read/water-farming/id/846
https://newsela.com/read/obama-king/id/971
https://newsela.com/read/yosemite-fire/id/955
https://newsel